In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sector/sector_unfiltered.csv


In [2]:
#!pip install transformers --upgrade

In [3]:
#!pip install peft datasets evaluate


In [4]:
#!pip install accelerate

In [5]:
#pip install evaluate


In [6]:
df = pd.read_csv("/kaggle/input/sector/sector_unfiltered.csv")
df.head()

,id,text,vecID,summary_article,label,Comments,title,Words Per Article
0,57749,", /PRNewswire/ -- Boston Scientific (NYSE: ) h...",{'$numberLong': '8674496742022912'},This is the second consecutive year Boston Sci...,['Medical Technology'],[],NaN,390
1,57750,"NEW YORK, Dec 22 (Reuters) - Nestle SA said on...",{'$numberLong': '8088413574956013'},”Bakus said Nestle had prioritized making the ...,['Food & Beverages Retail'],[],NaN,390
2,57751,"BERLIN/PARIS (Reuters) - Eat misshapen veg, wa...",{'$numberLong': '8045220543830533'},“It is far easier to get a consumer to switch ...,['Consumer Services'],[],NaN,1025
3,57752,Gift ArticleSharePope Francis has been called ...,{'$numberLong': '8011134654570867'},"“Sadly, even human rights can be used as a jus...",['None matched'],['this article is based on topic religion'],NaN,1406
4,57753,"Quick! What do Italian jackets, British beef,...",{'$numberLong': '895805351371523'},"Quick! What do Italian jackets, British beef,...",['Forestry & Paper'],[],NaN,527


In [7]:
df=df[['text','label']]

In [8]:
df.head()

,text,label
0,", /PRNewswire/ -- Boston Scientific (NYSE: ) h...",['Medical Technology']
1,"NEW YORK, Dec 22 (Reuters) - Nestle SA said on...",['Food & Beverages Retail']
2,"BERLIN/PARIS (Reuters) - Eat misshapen veg, wa...",['Consumer Services']
3,Gift ArticleSharePope Francis has been called ...,['None matched']
4,"Quick! What do Italian jackets, British beef,...",['Forestry & Paper']


In [9]:
len(df['label'].unique())

39

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df.head()

,text,label
0,", /PRNewswire/ -- Boston Scientific (NYSE: ) h...",27
1,"NEW YORK, Dec 22 (Reuters) - Nestle SA said on...",14
2,"BERLIN/PARIS (Reuters) - Eat misshapen veg, wa...",12
3,Gift ArticleSharePope Francis has been called ...,29
4,"Quick! What do Italian jackets, British beef,...",16


In [11]:
df=pd.DataFrame(df)

In [12]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset


Dataset({
    features: ['text', 'label'],
    num_rows: 6096
})

In [13]:
dataset.info.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [14]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification,DataCollatorWithPadding,TrainingArguments, Trainer

2024-03-12 04:33:28.082827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 04:33:28.082949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 04:33:28.215691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
model_id="roberta-base"
model=AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=len(df['label'].unique()))

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
tokenizer=AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
dataset=dataset.train_test_split(test_size=0.3)

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4267
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1829
    })
})

In [19]:
def preprocess(examples):
    tokenized=tokenizer(examples['text'],truncation=True,padding=True)
    return tokenized

In [20]:
tokenized_dataset=dataset.map(preprocess,batched=True,batch_size=None,remove_columns=["text"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
train_dataset=tokenized_dataset['train']
eval_dataset=tokenized_dataset['test'].shard(num_shards=2,index=0)
test_dataset=tokenized_dataset['test'].shard(num_shards=2,index=1)

In [22]:
print(train_dataset)
print('---------------------------------------')
print(eval_dataset)
print('---------------------------------------')
print(test_dataset)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4267
})
---------------------------------------
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 915
})
---------------------------------------
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 914
})


In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [27]:
#pip install evaluate

In [28]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy=acc_metric.compute(predictions=predictions, references=labels)
    #f1=f1_metric.compute(predictions=predictions, references=labels, average='micro')
    return accuracy

In [30]:
training_args = TrainingArguments(output_dir="Full_finetuning", evaluation_strategy="epoch",
                                 num_train_epochs=3,)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [32]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.895700,1.212343,0.690710
2,1.065500,1.078621,0.722404
3,0.713600,1.060528,0.723497


TrainOutput(global_step=1602, training_loss=1.1872872675253956, metrics={'train_runtime': 821.4537, 'train_samples_per_second': 15.583, 'train_steps_per_second': 1.95, 'total_flos': 3369203524555776.0, 'train_loss': 1.1872872675253956, 'epoch': 3.0})

In [33]:
test_prediction=trainer.predict(test_dataset)

In [35]:
preds=np.argmax(test_prediction.predictions,axis=-1)

In [55]:
#preds

In [56]:
#f1_metric._compute(predictions=preds, references=test_prediction.label_ids,average=None)#, average='micro'

In [41]:
acc_metric.compute(predictions=preds, references=test_prediction.label_ids)

{'accuracy': 0.6958424507658644}

In [47]:
from sklearn.metrics import classification_report

In [49]:
y_true =test_prediction.label_ids
y_pred=preds

In [53]:
y_true=label_encoder.inverse_transform(y_true)
y_pred=label_encoder.inverse_transform(y_pred)

In [54]:
print(classification_report(y_true, y_pred))

                                     precision    recall  f1-score   support

             ['Air Transportation']       0.83      1.00      0.90        19
             ['Alternative Energy']       0.61      0.68      0.64        25
             ['Apparel & Textiles']       0.91      0.93      0.92        46
                    ['Automobiles']       0.76      0.87      0.81        15
                      ['Beverages']       0.50      0.20      0.29         5
['Biotechnology & Pharmaceuticals']       0.69      0.66      0.67        44
                ['Capital Markets']       0.76      0.62      0.68        50
                      ['Chemicals']       1.00      0.67      0.80         6
                           ['Coal']       0.62      0.83      0.71         6
         ['Construction Materials']       0.00      0.00      0.00         5
['Consumer Discretionary Products']       0.75      0.66      0.70        32
          ['Consumer Goods Retail']       0.40      0.24      0.30        1

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
